In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
import time
from gerumo.data.dataset import describe_dataset
from gerumo.data.generators import build_generator
from gerumo.utils.engine import (
    setup_cfg, setup_environment, setup_experiment, setup_model,
    build_dataset, build_callbacks, build_metrics, build_optimizer, build_loss
)
from gerumo.models.base import build_model
from gerumo.visualization.metrics import training_history


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
args = dotdict()

## Select configuration

In [ ]:
args['config_file'] = '/home/asuka/projects/gerumo2/config/regression/cnn/cnn_lst.yml'
args['opts'] = [
    'SOLVER.BATCH_SIZE', 64,
    'GENERATOR.NAME', 'build_tf_record_dataset',
    'GENERATOR.KWARGS', [('tfRecords_folder', '/home/asuka/projects/gerumo2/notebooks/tfRecords/lst')],
    #'DATASETS.TRAIN.EVENTS', '/home/asuka/projects/gerumo2/datasets/gamma_sample/splits/validation_events.parquet',
    #'DATASETS.TRAIN.TELESCOPES', '/home/asuka/projects/gerumo2/datasets/gamma_sample/splits/validation_telescopes.parquet'
]

## Setup

In [ ]:
cfg = setup_cfg(args)
output_dir = setup_experiment(cfg)
logger = setup_environment(cfg)

## Load Datasets

In [ ]:
train_dataset = build_dataset(cfg, 'train')
describe_dataset(train_dataset, logger,
                save_to=output_dir / "train_description.txt")
validation_dataset = build_dataset(cfg, 'validation')
describe_dataset(validation_dataset, logger,
                save_to=output_dir / "validation_description.txt")

## Build generators

In [ ]:
train_generator, input_shape, train_size = build_generator(cfg, train_dataset, 'train')
validation_generator, _, validation_size = build_generator(cfg, validation_dataset, 'validation')

## Build model

In [ ]:
model = build_model(cfg, input_shape)

## Build training tools

In [ ]:
callbacks = build_callbacks(cfg)
metrics = build_metrics(cfg)
optimizer = build_optimizer(cfg, train_size)
loss = build_loss(cfg)

## Compile model

In [ ]:
model = setup_model(
    model, train_generator, optimizer, loss, metrics
)

## Start training

In [ ]:
model.fit_mode()

In [ ]:
start_time = time.time()
history = model.fit(
    train_generator,
    epochs= cfg.SOLVER.EPOCHS,
    steps_per_epoch=train_size,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=validation_size,
    callbacks=callbacks,
    use_multiprocessing=False,
    workers=5,
    max_queue_size=20,
)
training_time = (time.time() - start_time)/60.0

In [ ]:
logger.info(f'Training time: {training_time:.3f} [min]')

In [ ]:
training_history(history, training_time, cfg.EXPERIMENT_NAME)

In [ ]:
training_history(history, training_time, cfg.EXPERIMENT_NAME, ylog=True)